Para esta función vamos a precisar 2 de los dataframes limpios. El dataframe dfSteamCleaned para obtener la información sobre el precio de los juegos, el dataframe user_reviews_df para obtener el id de usuario y la cantidad de items que tiene y el porcentaje de recomendaciones.

In [1]:
import pandas as pd
import numpy as np

C:\Users\kanza\AppData\Local\Temp\ipykernel_9408\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Abrimos los dataframes

In [2]:
dfSteam = pd.read_csv('../dfSteamCleaned.csv')
dfUserReviews = pd.read_csv('../user_reviews_df_cleaned.csv')

C:\Users\kanza\AppData\Local\Temp\ipykernel_9408\1071469300.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dfSteam = pd.read_csv('../dfSteamCleaned.csv')


In [3]:
dfSteam.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
dfUserReviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,2
3,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,2
4,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,2


Del dataframe dfSteam vamos a conservar solamente las columnas id y price. Y del dataframe dfUserRreviews vamos a conservar user_id, item_id y recommend.

In [5]:
dfSteam = dfSteam[['id','price']]
dfUserReviews = dfUserReviews[['user_id','item_id','recommend']]

Verificamos si los campos de los id tienen valores nulos. También eliminamos a los usuarios que no tengan ningún item.

In [6]:
dfSteam[(dfSteam['id'].isna())]

,id,price
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
88306,NaN,NaN
88307,NaN,NaN
88308,NaN,NaN
88383,NaN,19.99


In [7]:
dfUserReviews[(dfUserReviews['item_id'].isna())]

,user_id,item_id,recommend
137,gdxsd,NaN,NaN
177,76561198094224872,NaN,NaN
2559,76561198021575394,NaN,NaN
10080,cmuir37,NaN,NaN
13767,Jaysteeny,NaN,NaN
15493,ML8989,NaN,NaN
19184,76561198079215291,NaN,NaN
20223,76561198079342142,NaN,NaN
25056,76561198061996985,NaN,NaN
26257,76561198108286351,NaN,NaN


In [8]:
dfUserReviews.dropna(subset='item_id', inplace=True)

In [9]:
dfSteam.dropna(how='all', inplace=True)

Hay dos items que no tienen id y no podremos asociarlos a ningún usuario

In [10]:
dfSteam[(dfSteam['id'].isna())]

,id,price
88383,NaN,19.99
119270,NaN,19.99


In [11]:
dfSteam.dropna(subset='id', inplace=True)

Convertimos los id a enteros

In [12]:
dfSteam['id'] = dfSteam['id'].astype(int)
dfUserReviews['item_id'] = dfUserReviews['item_id'].astype(int)

Ahora unimos los dataframes haciendo coincidir dfSteam['id'] con dfUserReviews['item_id']

In [13]:
df = pd.merge(dfUserReviews,dfSteam,left_on='item_id', right_on='id')

In [14]:
del(dfUserReviews,dfSteam)

In [15]:
df

,user_id,item_id,recommend,id,price
0,76561197970982479,1250,True,1250,19.99
1,76561197970982479,22200,True,22200,9.99
2,js41637,227300,True,227300,19.99
3,js41637,239030,True,239030,9.99
4,evcentric,248820,True,248820,9.99
...,...,...,...,...,...
53983,76561198312638244,70,True,70,9.99
53984,76561198312638244,362890,True,362890,19.99
53985,LydiaMorley,273110,True,273110,Free to Play
53986,LydiaMorley,730,True,730,14.99


Vemos que hay precios que no están en formato numérico. Vamos a detectarlos y reemplazarlos por 0

In [16]:
df['price'].unique()

array(['19.99', '9.99', '6.99', '14.99', '24.99', 'Free to Play',
       'Free To Play', '29.99', '4.99', 'Free', '39.99', '2.99', '34.99',
       '0.99', nan, '17.99', '3.99', '1.99', '7.99', '11.99', '59.99',
       '15.99', '49.99', '3.49', '5.99', '12.99', '59.95', '13.98',
       '74.76', 'Play for Free!', '10.99', 'Free Movie', '16.99', '8.99',
       '44.99', '13.99', '69.99', '54.99', '20.0', '160.91', '0.49',
       '16.06', '7.0', '18.99', 'Free HITMAN™ Holiday Pack', '23.96',
       '14.95', '15.0', '19.98', '3.0', '13.37', 'Third-party', '79.99',
       '7.49', '771.71', '21.99', '12.89', '139.92', 'Free Mod', '189.96',
       '1.87', '99.99', '32.99', '5.49', 'Free to Use', 'Install Now',
       '0.98', '23.99', 'Play WARMACHINE: Tactics Demo', 'Play the Demo'],
      dtype=object)

In [17]:
type(df['price'][0])

str

Si intentamos convertir los precios a float obtendremos un error debido a los valores textuales. Para solucionar ese problema creamos las funciones conversor y reemplazo. La primera intenta convertir los valores con un bloque try except, si no puede hacerlo, agrega el valor a la lsita valoresTextuales.
La segunda itera sobre la columna con los precios convertidos y si el valor se encuentra en la lsita valoresTextuales lo reemplaza por cero.

In [18]:
valoresTextuales=[]

In [19]:
def conversor(x):
    try:
        x=float(x)
    except:
        if x not in valoresTextuales:
            valoresTextuales.append(x)        
    return x

In [20]:
df['price']=df['price'].apply(conversor)

In [21]:
valoresTextuales

['Free to Play',
 'Free To Play',
 'Free',
 'Play for Free!',
 'Free Movie',
 'Free HITMAN™ Holiday Pack',
 'Third-party',
 'Free Mod',
 'Free to Use',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Play the Demo']

In [22]:
df['price'] = df['price'].fillna(0)

In [23]:
def reemplazo(x):
    if x in valoresTextuales:
        x=0
    return x

In [24]:
df['price'] = df['price'].apply(reemplazo)

Vemos que desaparecieron los valores textuales y nulos. Si ejecutamos las celdas para ver los valores únicos lo comprobaremos.

In [25]:
df

,user_id,item_id,recommend,id,price
0,76561197970982479,1250,True,1250,19.99
1,76561197970982479,22200,True,22200,9.99
2,js41637,227300,True,227300,19.99
3,js41637,239030,True,239030,9.99
4,evcentric,248820,True,248820,9.99
...,...,...,...,...,...
53983,76561198312638244,70,True,70,9.99
53984,76561198312638244,362890,True,362890,19.99
53985,LydiaMorley,273110,True,273110,0.00
53986,LydiaMorley,730,True,730,14.99


Eliminaremos las variables, funciones, datframes, etc. que no utilicemos para liberar la mayor cantidad de recursos prosibles.

In [26]:
del(valoresTextuales,conversor,reemplazo)

Crearemos 3 dataframes, todos agrupados por usuarios, y con las funciones de agregación contaremos los items en el dataframe item_count, calcularemos el total del gasto de los items en el dataframe total_gastado y por último dividiremos la cantidad de recomendaciones verdaderas en el total de recomendaciones de cada usuario para calcular el porcentaje.

In [27]:
item_count = df.groupby('user_id')['item_id'].count().reset_index(name='items_count')

In [28]:
item_count

,user_id,items_count
0,--000--,1
1,--ace--,2
2,--ionex--,2
3,-2SV-vuLB-Kg,4
4,-Azsael-,1
...,...,...
24375,zvanik,4
24376,zwanzigdrei,1
24377,zy0705,1
24378,zynxgameth,1


In [29]:
total_gastado = df.groupby('user_id')['price'].sum().reset_index(name='Total Gastado')

In [30]:
total_gastado

,user_id,Total Gastado
0,--000--,19.99
1,--ace--,4.99
2,--ionex--,24.98
3,-2SV-vuLB-Kg,64.97
4,-Azsael-,29.99
...,...,...
24375,zvanik,14.99
24376,zwanzigdrei,0.00
24377,zy0705,0.00
24378,zynxgameth,0.00


In [31]:
porcentaje_recomendacion = df.groupby('user_id')['recommend'].sum()/df.groupby('user_id')['recommend'].count()*100
porcentaje_recomendacion = porcentaje_recomendacion.reset_index(name='recommend')

In [32]:
porcentaje_recomendacion[(porcentaje_recomendacion['recommend'])!=100]

,user_id,recommend
9,-PRoSlayeR-,50.0
10,-SEVEN-,77.777778
15,-_PussyDestroyer_-,75.0
21,00454211432342,50.0
29,01189958889189157253,80.0
...,...,...
24340,zile98,66.666667
24346,zixwot,66.666667
24364,zp3413,66.666667
24373,zuzuga2003,75.0


Finalmente, unimos todos los dataframes en uno solo

In [33]:
resultados= pd.merge(item_count, total_gastado, on='user_id')

In [34]:
resultados = pd.merge(resultados, porcentaje_recomendacion, on='user_id')

In [35]:
resultados['recommend']=resultados['recommend'].apply(lambda x: round(x,2))

In [36]:
resultados[(resultados['recommend']!=100)]

,user_id,items_count,Total Gastado,recommend
9,-PRoSlayeR-,2,5.98,50.00
10,-SEVEN-,9,151.91,77.78
15,-_PussyDestroyer_-,4,17.98,75.00
21,00454211432342,2,14.99,50.00
29,01189958889189157253,5,74.95,80.00
...,...,...,...,...
24340,zile98,6,69.96,66.67
24346,zixwot,3,64.97,66.67
24364,zp3413,3,19.99,66.67
24373,zuzuga2003,4,29.97,75.00


Guardamos los resultados en un archivo csv para la api

In [37]:
resultados.to_csv('userDataResults.csv', index=False)

In [38]:
del(df,item_count,total_gastado,porcentaje_recomendacion,resultados)